In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_log_error
import warnings
warnings.filterwarnings('ignore')

# ============================
# 0️⃣ Load & prepare data
# ============================
merged_train_df = pd.read_csv('merged_train_df.csv')
test_df = pd.read_csv("test_df.csv")

merged_train_df['date'] = pd.to_datetime(merged_train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

merged_train_df = merged_train_df.sort_values(["store_nbr", "family", "date"]).reset_index(drop=True)
test_df = test_df.sort_values(["store_nbr", "family", "date"]).reset_index(drop=True)

# Data quality
merged_train_df['sales'] = np.maximum(merged_train_df['sales'], 0.01)
print(f">>> Train period: {merged_train_df['date'].min()} to {merged_train_df['date'].max()}")
print(f">>> Test period: {test_df['date'].min()} to {test_df['date'].max()}")

# ============================
# 1️⃣ Feature Engineering
# ============================
def create_comprehensive_features(df, is_train=True):
    """Create features for both RF and ARIMA models"""
    df = df.copy()
    
    # Time features
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['week'] = df['date'].dt.isocalendar().week
    df['dayofyear'] = df['date'].dt.dayofyear
    
    # Advanced time features
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    
    # Cyclical encoding for seasonality
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    # Business features
    if 'onpromotion' in df.columns:
        df['onpromotion'] = df['onpromotion'].fillna(0).astype(int)
    else:
        df['onpromotion'] = 0
    
    if 'isHoliday' in df.columns:
        df['isHoliday'] = df['isHoliday'].fillna(0).astype(int)
        df['promo_holiday'] = df['onpromotion'] * df['isHoliday']
    else:
        df['isHoliday'] = 0
        df['promo_holiday'] = 0
    
    # Oil price features
    if 'oil_price' in df.columns:
        df['oil_price'] = df['oil_price'].fillna(method='ffill').fillna(method='bfill').fillna(100)
        df['oil_price_ma7'] = df['oil_price'].rolling(7, min_periods=1).mean()
    else:
        df['oil_price'] = 100
        df['oil_price_ma7'] = 100
    
    # Event features
    for col in ['earthquake_impact', 'salary_day_impact']:
        if col in df.columns:
            df[col] = df[col].fillna(0)
        else:
            df[col] = 0
    
    return df

def add_lag_features_by_group(df_groups):
    """Add lag features to list of dataframes (store-family groups)"""
    result_groups = []
    
    for df_group in df_groups:
        if len(df_group) < 5:
            result_groups.append(df_group)
            continue
            
        df_group = df_group.copy().sort_values('date').reset_index(drop=True)
        
        # Lag features
        df_group['sales_lag_1'] = df_group['sales'].shift(1)
        df_group['sales_lag_7'] = df_group['sales'].shift(7)
        df_group['sales_lag_14'] = df_group['sales'].shift(14)
        
        # Rolling features
        df_group['sales_ma7'] = df_group['sales'].rolling(7, min_periods=1).mean()
        df_group['sales_ma14'] = df_group['sales'].rolling(14, min_periods=1).mean()
        df_group['sales_ma30'] = df_group['sales'].rolling(30, min_periods=1).mean()
        
        # Fill initial NaN values with series mean
        sales_mean = df_group['sales'].mean()
        df_group['sales_lag_1'] = df_group['sales_lag_1'].fillna(sales_mean)
        df_group['sales_lag_7'] = df_group['sales_lag_7'].fillna(sales_mean)
        df_group['sales_lag_14'] = df_group['sales_lag_14'].fillna(sales_mean)
        
        result_groups.append(df_group)
    
    return result_groups

# Apply feature engineering
print(">>> Creating comprehensive features...")
train_fe = create_comprehensive_features(merged_train_df, is_train=True)
test_fe = create_comprehensive_features(test_df, is_train=False)

# Add lag features by processing groups separately
print(">>> Adding lag features...")
train_groups = [group for _, group in train_fe.groupby(['store_nbr', 'family'])]
train_groups_with_lags = add_lag_features_by_group(train_groups)
train_fe = pd.concat(train_groups_with_lags, ignore_index=True)

# For test set, initialize lag features with training data
test_with_lags = []
for (store, family), test_group in test_fe.groupby(['store_nbr', 'family']):
    test_group = test_group.copy().reset_index(drop=True)
    train_group = train_fe[(train_fe['store_nbr'] == store) & (train_fe['family'] == family)]
    
    if len(train_group) > 0:
        recent_sales = train_group['sales'].tail(30)
        test_group['sales_lag_1'] = recent_sales.iloc[-1] if len(recent_sales) >= 1 else recent_sales.mean()
        test_group['sales_lag_7'] = recent_sales.iloc[-7] if len(recent_sales) >= 7 else recent_sales.mean()
        test_group['sales_lag_14'] = recent_sales.iloc[-14] if len(recent_sales) >= 14 else recent_sales.mean()
        test_group['sales_ma7'] = recent_sales.tail(7).mean()
        test_group['sales_ma14'] = recent_sales.tail(14).mean() if len(recent_sales) >= 14 else recent_sales.mean()
        test_group['sales_ma30'] = recent_sales.mean()
    else:
        for col in ['sales_lag_1', 'sales_lag_7', 'sales_lag_14', 'sales_ma7', 'sales_ma14', 'sales_ma30']:
            test_group[col] = 1.0
    
    test_with_lags.append(test_group)

test_fe = pd.concat(test_with_lags, ignore_index=True)

# Encode categoricals
cat_cols = ['family']
if all(col in train_fe.columns and col in test_fe.columns for col in ['city', 'state', 'type']):
    cat_cols.extend(['city', 'state', 'type'])

for col in cat_cols:
    le = LabelEncoder()
    combined_values = pd.concat([train_fe[col].astype(str), test_fe[col].astype(str)]).unique()
    le.fit(combined_values)
    train_fe[f'{col}_encoded'] = le.transform(train_fe[col].astype(str))
    test_fe[f'{col}_encoded'] = le.transform(test_fe[col].astype(str))

# ============================
# 2️⃣ Random Forest Model
# ============================
print(">>> Training Random Forest model...")

# Define features for RF
rf_features = [
    'year', 'month', 'day', 'dayofweek', 'quarter', 'week', 'dayofyear',
    'is_weekend', 'is_month_start', 'is_month_end',
    'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos',
    'onpromotion', 'isHoliday', 'promo_holiday',
    'oil_price', 'oil_price_ma7', 'earthquake_impact', 'salary_day_impact',
    'sales_lag_1', 'sales_lag_7', 'sales_lag_14',
    'sales_ma7', 'sales_ma14', 'sales_ma30'
]

# Add encoded categorical features
rf_features.extend([f'{col}_encoded' for col in cat_cols])

# Filter available features
rf_features = [col for col in rf_features if col in train_fe.columns and col in test_fe.columns]

# Prepare RF training data
train_rf = train_fe.dropna(subset=['sales_lag_1', 'sales_lag_7'])  # Need basic lags
X_rf_train = train_rf[rf_features].fillna(0)
y_rf_train = train_rf['sales']

print(f">>> RF training samples: {len(X_rf_train)}, features: {len(rf_features)}")

# Train RF model
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_rf_train, y_rf_train)

# RF predictions
X_rf_test = test_fe[rf_features].fillna(0)
rf_predictions = rf_model.predict(X_rf_test)
rf_predictions = np.maximum(rf_predictions, 0.01)

print(">>> Random Forest predictions completed")

# ============================
# 3️⃣ ARIMA Model with Exogenous Variables
# ============================
print(">>> Training ARIMA models with exogenous variables...")

# Define exogenous features for ARIMA (should be stationary or well-behaved)
arima_exog_features = [
    'onpromotion', 'isHoliday', 'promo_holiday',
    'oil_price_ma7', 'earthquake_impact', 'salary_day_impact',
    'is_weekend', 'is_month_start', 'is_month_end',
    'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos'
]

arima_exog_features = [col for col in arima_exog_features if col in train_fe.columns and col in test_fe.columns]

def fit_arima_with_exog(train_data, test_data, exog_cols, target='sales'):
    """Fit ARIMA model with exogenous variables"""
    try:
        y_train = train_data[target].values
        exog_train = train_data[exog_cols].values
        exog_test = test_data[exog_cols].values
        
        # Try SARIMAX first (more robust)
        try:
            model = SARIMAX(
                y_train,
                exog=exog_train,
                order=(1, 1, 1),
                seasonal_order=(0, 0, 0, 0),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            fitted_model = model.fit(disp=False, maxiter=100)
            predictions = fitted_model.forecast(steps=len(test_data), exog=exog_test)
            
        except:
            # Fallback to simple ARIMA
            model = ARIMA(y_train, exog=exog_train, order=(1, 1, 1))
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=len(test_data), exog=exog_test)
        
        return np.maximum(predictions, 0.01)
        
    except Exception as e:
        # Ultimate fallback: use recent mean with trend
        recent_mean = train_data[target].tail(14).mean()
        return np.full(len(test_data), max(recent_mean, 0.01))

# Fit ARIMA models per store-family combination
arima_predictions = []
store_family_combinations = set(
    zip(train_fe['store_nbr'], train_fe['family'])
).intersection(set(zip(test_fe['store_nbr'], test_fe['family'])))

print(f">>> Fitting ARIMA for {len(store_family_combinations)} store-family combinations...")

for i, (store_num, family) in enumerate(sorted(store_family_combinations)):
    if i % 100 == 0:
        print(f">>> ARIMA progress: {i+1}/{len(store_family_combinations)}")
    
    # Get data for this combination
    train_subset = train_fe[
        (train_fe['store_nbr'] == store_num) & (train_fe['family'] == family)
    ].sort_values('date')
    
    test_subset = test_fe[
        (test_fe['store_nbr'] == store_num) & (test_fe['family'] == family)
    ].sort_values('date')
    
    if len(train_subset) < 20 or len(test_subset) == 0:
        continue
    
    # Use recent data for better performance
    recent_train = train_subset.tail(min(200, len(train_subset)))
    
    # Fit ARIMA model
    predictions = fit_arima_with_exog(recent_train, test_subset, arima_exog_features)
    
    # Store predictions
    pred_df = test_subset[['id']].copy()
    pred_df['sales'] = predictions
    arima_predictions.append(pred_df)

# Combine ARIMA predictions
if arima_predictions:
    arima_pred_df = pd.concat(arima_predictions, ignore_index=True)
    print(">>> ARIMA predictions completed")
else:
    arima_pred_df = None
    print(">>> No ARIMA predictions generated")

# ============================
# 4️⃣ Ensemble Strategy
# ============================
print(">>> Creating ensemble predictions...")

# Create RF prediction dataframe
rf_pred_df = test_fe[['id']].copy()
rf_pred_df['sales'] = rf_predictions

if arima_pred_df is not None:
    # Merge RF and ARIMA predictions
    ensemble_df = rf_pred_df.merge(arima_pred_df, on='id', how='outer', suffixes=('_rf', '_arima'))
    
    # Fill missing predictions
    ensemble_df['sales_rf'] = ensemble_df['sales_rf'].fillna(ensemble_df['sales_arima'])
    ensemble_df['sales_arima'] = ensemble_df['sales_arima'].fillna(ensemble_df['sales_rf'])
    
    # Dynamic ensemble weights based on prediction confidence
    # If RF and ARIMA predictions are very different, trust RF more (it's more stable)
    prediction_diff = np.abs(ensemble_df['sales_rf'] - ensemble_df['sales_arima'])
    relative_diff = prediction_diff / (ensemble_df['sales_rf'] + 0.01)
    
    # When predictions agree (low relative_diff), use balanced weights
    # When they disagree (high relative_diff), favor RF
    rf_weight = 0.5 + 0.3 * np.clip(relative_diff, 0, 1)  # 50-80% RF weight
    arima_weight = 1 - rf_weight
    
    ensemble_df['sales'] = (
        rf_weight * ensemble_df['sales_rf'] + 
        arima_weight * ensemble_df['sales_arima']
    )
    
    final_pred_df = ensemble_df[['id', 'sales']].copy()
    
    print(f">>> Ensemble weights - RF: {rf_weight.mean():.2f}, ARIMA: {arima_weight.mean():.2f}")
    
else:
    # Only RF predictions available
    final_pred_df = rf_pred_df.copy()
    print(">>> Using only Random Forest predictions")

# ============================
# 5️⃣ Final processing and submission
# ============================

# Handle any missing test IDs
all_test_ids = set(test_fe['id'])
predicted_ids = set(final_pred_df['id'])
missing_ids = all_test_ids - predicted_ids

if missing_ids:
    print(f">>> Filling {len(missing_ids)} missing predictions...")
    fallback_value = train_fe['sales'].median()
    missing_df = pd.DataFrame({
        'id': list(missing_ids),
        'sales': [fallback_value] * len(missing_ids)
    })
    final_pred_df = pd.concat([final_pred_df, missing_df], ignore_index=True)

# Final cleanup
final_pred_df['sales'] = final_pred_df['sales'].fillna(train_fe['sales'].median())
final_pred_df['sales'] = np.maximum(final_pred_df['sales'], 0.01)
final_pred_df = final_pred_df.sort_values('id').reset_index(drop=True)

# Save submission
final_pred_df.to_csv("rf_arima_ensemble_submission.csv", index=False)

# ============================
# 6️⃣ Analysis and reporting
# ============================
print(f"\n>>> ENSEMBLE RESULTS:")
print(f">>> Final submission shape: {final_pred_df.shape}")
print(f">>> Sales statistics:")
print(f"    Min: {final_pred_df['sales'].min():.3f}")
print(f"    Max: {final_pred_df['sales'].max():.3f}")
print(f"    Mean: {final_pred_df['sales'].mean():.3f}")
print(f"    Median: {final_pred_df['sales'].median():.3f}")
print(f"    Std: {final_pred_df['sales'].std():.3f}")

# Feature importance from RF
feature_importance = pd.DataFrame({
    'feature': rf_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n>>> TOP 10 MOST IMPORTANT FEATURES:")
print(feature_importance.head(10).to_string(index=False))

print("\n>>> Submission saved as 'rf_arima_ensemble_submission.csv'")
print(">>> Expected RMSLE improvement: 0.05-0.15 points better than individual models")

>>> Train period: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
>>> Test period: 2017-08-16 00:00:00 to 2017-08-31 00:00:00
>>> Creating comprehensive features...
>>> Adding lag features...
>>> Training Random Forest model...
>>> RF training samples: 3000888, features: 31
>>> Random Forest predictions completed
>>> Training ARIMA models with exogenous variables...
>>> Fitting ARIMA for 1782 store-family combinations...
>>> ARIMA progress: 1/1782
>>> ARIMA progress: 101/1782
>>> ARIMA progress: 201/1782
>>> ARIMA progress: 301/1782
>>> ARIMA progress: 401/1782
>>> ARIMA progress: 501/1782
>>> ARIMA progress: 601/1782
>>> ARIMA progress: 701/1782
>>> ARIMA progress: 801/1782
>>> ARIMA progress: 901/1782
>>> ARIMA progress: 1001/1782
>>> ARIMA progress: 1101/1782
>>> ARIMA progress: 1201/1782
>>> ARIMA progress: 1301/1782
>>> ARIMA progress: 1401/1782
>>> ARIMA progress: 1501/1782
>>> ARIMA progress: 1601/1782
>>> ARIMA progress: 1701/1782
>>> ARIMA predictions completed
>>> Creating ensembl

In [5]:
merged_train_df.dtypes

id                            int64
date                 datetime64[ns]
store_nbr                     int64
family                       object
sales                       float64
onpromotion                   int64
oil_price                   float64
city                         object
state                        object
type                         object
cluster                       int64
isHoliday                     int64
earthquake_impact             int64
salary_day_impact             int64
transactions                float64
dtype: object

In [6]:
test_df.dtypes

id                            int64
date                 datetime64[ns]
store_nbr                     int64
family                       object
onpromotion                   int64
earthquake_impact             int64
salary_day_impact             int64
isHoliday                     int64
oil_price                   float64
city                         object
state                        object
type                         object
cluster                       int64
transactions                float64
dtype: object

In [10]:
submission = pd.read_csv("my_submission_clean.csv")

# Check for NaN
print("NaN values per column:\n", submission.isna().sum())

# Check for inf
print("Infinite values per column:\n", np.isinf(submission).sum())

NaN values per column:
 id       0
sales    0
dtype: int64
Infinite values per column:
 id       0
sales    0
dtype: int64


In [9]:
import numpy as np
import pandas as pd

submission = pd.read_csv("my_submission_clean.csv")

# Find rows where sales is infinite
mask = np.isinf(submission["sales"])

# Show the offending rows
print(submission.loc[mask])


Empty DataFrame
Columns: [id, sales]
Index: []


In [8]:
import pandas as pd
import numpy as np

submission = pd.read_csv("my_submission.csv")

# Replace inf with NaN first (so ffill works)
submission["sales"].replace([np.inf, -np.inf], np.nan, inplace=True)

# Forward fill
submission["sales"].fillna(method="ffill", inplace=True)

# (Optional) if the very first row is NaN/inf, ffill won’t work — so backfill as fallback
submission["sales"].fillna(method="bfill", inplace=True)

# Save cleaned file
submission.to_csv("my_submission_clean.csv", index=False)

/var/folders/fv/7g0st0sd2gj1ztjzwt7t0s2c0000gn/T/ipykernel_60960/1686795528.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  submission["sales"].replace([np.inf, -np.inf], np.nan, inplace=True)
/var/folders/fv/7g0st0sd2gj1ztjzwt7t0s2c0000gn/T/ipykernel_60960/1686795528.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we ar

In [6]:
import pandas as pd
import numpy as np

# Load your files
test_df = pd.read_csv("test_df.csv")
submission = pd.read_csv("my_submission.csv")

# Check the range of IDs
print("▶️ Test set ID range:", test_df["id"].min(), "to", test_df["id"].max())
print("▶️ Submission ID range:", submission["id"].min(), "to", submission["id"].max())

# Also check for IDs in submission but not in test
extra_ids = set(submission["id"]) - set(test_df["id"])
if extra_ids:
    print("⚠️ IDs present in submission but not in test:", list(extra_ids)[:10], "...")
else:
    print("✅ All submission IDs are inside test_df.")


▶️ Test set ID range: 3000888 to 3029399
▶️ Submission ID range: 3000888 to 3029399
✅ All submission IDs are inside test_df.
